In [2]:
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.3 MB/s eta 0:00:0000:010:01

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install wordcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.1/511.1 kB 7.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import os
import re
import pickle
import string
import unicodedata
from random import randint

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from wordcloud import STOPWORDS, WordCloud

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Embedding, TimeDistributed

E0000 00:00:1731762195.130800      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D1116 13:03:15.138876195      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D1116 13:03:15.138889414      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D1116 13:03:15.138892708      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D1116 13:03:15.138895046      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D1116 13:03:15.138897457      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defa

In [5]:
!pip install contractions==0.1.73

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 11.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
from contractions import contractions_dict

for key, value in list(contractions_dict.items())[:10]:
    print(f'{key} == {value}')

I'm == I am
I'm'a == I am about to
I'm'o == I am going to
I've == I have
I'll == I will
I'll've == I will have
I'd == I would
I'd've == I would have
Whatcha == What are you
amn't == am not


In [7]:
# Using TPU

# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1731762211.146862      13 service.cc:145] XLA service 0x59bffc32cf30 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731762211.146936      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1731762211.146941      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1731762211.146944      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1731762211.146947      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1731762211.146950      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1731762211.146953      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1731762211.146955      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1731762211.146963      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:6, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


Loading data

In [8]:
train_data = pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv")

In [9]:
train_data

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...
...,...,...,...
287108,fffdfb56fdf1a12d364562cc2b9b1d4de7481dee,By . James Rush . Former first daughter Chelse...,Chelsea Clinton said question of running for o...
287109,fffeecb8690b85de8c3faed80adbc7a978f9ae2a,An apologetic Vanilla Ice has given his first ...,"Vanilla Ice, 47 - real name Robert Van Winkle ..."
287110,ffff5231e4c71544bc6c97015cdb16c60e42b3f4,America's most lethal sniper claimed he wished...,America's most lethal sniper made comment in i...
287111,ffff924b14a8d82058b6c1c5368ff1113c1632af,"By . Sara Malm . PUBLISHED: . 12:19 EST, 8 Mar...",A swarm of more than one million has crossed b...


In [10]:
test_data = pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv")

In [11]:
test_data

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."
...,...,...,...
11485,ed8674cc15b29a87d8df8de1efee353d71122272,Our young Earth may have collided with a body ...,Oxford scientists say a Mercury-like body stru...
11486,2f58d1a99e9c47914e4b1c31613e3a041cd9011e,A man facing trial for helping his former love...,Man accused of helping former lover kill woman...
11487,411f6d57825161c3a037b4742baccd6cd227c0c3,A dozen or more metal implements are arranged ...,Marianne Power tried the tuning fork facial at...
11488,b5683ef8342056b17b068e0d59bdbe87e3fe44ea,Brook Lopez dominated twin brother Robin with ...,Brooklyn Nets beat the Portland Trail Blazers ...


In [12]:
val_data = pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv")

In [13]:
val_data

,id,article,highlights
0,61df4979ac5fcc2b71be46ed6fe5a46ce7f071c3,"Sally Forrest, an actress-dancer who graced th...","Sally Forrest, an actress-dancer who graced th..."
1,21c0bd69b7e7df285c3d1b1cf56d4da925980a68,A middle-school teacher in China has inked hun...,Works include pictures of Presidential Palace ...
2,56f340189cd128194b2e7cb8c26bb900e3a848b4,A man convicted of killing the father and sist...,"Iftekhar Murtaza, 29, was convicted a year ago..."
3,00a665151b89a53e5a08a389df8334f4106494c2,Avid rugby fan Prince Harry could barely watch...,Prince Harry in attendance for England's crunc...
4,9f6fbd3c497c4d28879bebebea220884f03eb41a,A Triple M Radio producer has been inundated w...,Nick Slater's colleagues uploaded a picture to...
...,...,...,...
13363,e93f721ba4949f21f33549c4a21d55ff456af979,All shops will be allowed to offer ‘click and ...,Shops won't have to apply for planning permiss...
13364,8df19a570ad14119a7d00f3bbe864fedf8c1691d,Mo Farah has had his nationality called into q...,Mo Farah broke the European half-marathon reco...
13365,2fdd5f89aa26e91ceea9b0ef264abfcfc3e6fa2e,Wolves kept their promotion hopes alive with a...,Wolves are three points off the play-off place...
13366,530d7b18d7a715b368b0745f9dfebfe353adeda8,A Brown University graduate student has died ...,"Hyoun Ju Sohn, a 25-year-old doctoral student,..."


In [14]:
def expand_contractions(text, contraction_map=contractions_dict):
    # Using regex for getting all contracted words
    contractions_keys = '|'.join(contraction_map.keys())
    contractions_pattern = re.compile(f'({contractions_keys})', flags=re.DOTALL)

    def expand_match(contraction):
        # Getting entire matched sub-string
        match = contraction.group(0)
        expanded_contraction = contraction_map.get(match)
        if not expand_contractions:
            print(match)
            return match
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text


expand_contractions("y'all can't expand contractions i'd think")

'you all cannot expand contractions id think'

In [15]:
# Converting to lowercase
train_data.highlights = train_data.highlights.apply(str.lower)
train_data.article = train_data.article.apply(str.lower)

train_data.sample(5)

,id,article,highlights
255294,d674fae658bce4c00efc4532b730ed40a6212686,"author] . published: . 05:22 est, 27 june 2013...","elena raouna, 22, who lives in doncaster, won ..."
45548,8062294af7a75010032ebce12da104c974a4b8ff,by . daniel miller . last updated at 1:38 am o...,fears that growth of internet dating is puttin...
144320,46a1ed33952671673a33f35c6e4afbaa4ed06506,by . daily mail reporter . published: . 21:25 ...,cameron bay says she wishes she had pushed for...
13749,26f724dc90858edfd0c4116c75b53b9b4d81d4cc,if you struggle creating that perfect feline f...,charlotte tilbury unveils £200 box with make-u...
122439,2a3edf8ebdb50b066b32a1c66e31d30282dc6ff6,"a gunman allegedly killed his adoptive mother,...","john lee, 29, is suspected of triple murder an..."


In [16]:
# Converting to lowercase
test_data.highlights = test_data.highlights.apply(str.lower)
test_data.article = test_data.article.apply(str.lower)

test_data.sample(5)

,id,article,highlights
2579,169f2f64514bd6b5d739e74d97eb1124fa0e0d33,"they were the biggest of the ice age giants, r...",the last mammoths died out on an arctic island...
2872,b092d88c11f3d2bf42ccead3d19402ca40b166ae,passengers hoping to make a short domestic fli...,"flight from kansas city, missouri to denver, c..."
605,b42bce47f5de4ba0ed5029c00fde475836b86799,"kellie maloney, pictured at ukip's spring conf...",star formerly known as frank is one of the par...
5151,66c25bf3c0647a8e306b1f5300cadf55429e119a,arsenal star alexis sanchez says he is 'very p...,arsenal's alexis sanchez is enjoying a fine de...
6675,813e85c8caa0290ecb543d95cb34b5d0a3ef77a3,google is facing a fine of up to £4billion as ...,eu has been investigating the us search giant'...


In [17]:
# Converting to lowercase
val_data.highlights = val_data.highlights.apply(str.lower)
val_data.article = val_data.article.apply(str.lower)

val_data.sample(5)

,id,article,highlights
983,3bdd0b1b14b8fbbc725ce7c2d8f01e679e8096ae,barcelona booked their place in the copa del r...,"barcelona beat villarreal 3-1 on wednesday, pr..."
10844,c86d503af0d1daa5782a2fecbedf2e594c3d7fb2,hong kong (cnn)the search is on for china's te...,"china wants 1,000 male junior high school stud..."
8190,f11ccf970d6f2d4d9c5539be27f4735da6da62e1,cristiano ronaldo has been said to have had a ...,cristiano ronaldo took to instagram to showcas...
6938,814196b6bbfc024f38dd54673367355ae736e94e,gareth bale has been taking more than his fair...,real madrid forward gareth bale took part in t...
5554,0366a9353b6675b3f5c95d16ee4a7bae5d7ea10d,talk about taking winter sports to the next le...,"hundreds of daredevil snow boarders, and their..."


In [20]:
#expanding contractions
train_data.highlights = train_data.highlights.apply(expand_contractions)
train_data.article = train_data.article.apply(expand_contractions)
train_data.sample(5)

,id,article,highlights
127587,30ea5659ebc013d17a8eb0fe4f2083327e832b82,wojciech szczesny and a selection of other ars...,wojciech szczesny helped gunners claim 2-1 vic...
26704,4bd1a0ce7efd46d6b74d47771889b37f5b15ce7e,"by . emily davies . published: . 12:43 est, 1 ...",platinum dummy is worth more than five times t...
4769,0dae469a260ba74a0adaa88c861e796135af50c3,the former residence of the late australian au...,the woollahra terrace was last bought by sir h...
273621,ee73253e76f15502e9793baab72dad98e0d0b4b1,danny cipriani has no intention of giving up. ...,danny cipriani was left out of englands autumn...
40345,71dea8361730c8d4bc30cbc4490c4f6e71aa8ea3,a spanish politician has been busted staring a...,cantabria politician seen looking at naked wom...


In [21]:
#expanding contractions
test_data.highlights = test_data.highlights.apply(expand_contractions)
test_data.article = test_data.article.apply(expand_contractions)
test_data.sample(5)

,id,article,highlights
1579,2586dff7ef4374a9bd4a0cf94a76c009d235b58a,a coroner has warned of the dangers of bedside...,seven-week-old grace roseman died in adjustabl...
7285,b720f3f83c700fc13483f9cbf4ee0d44f581fa17,jacques burger encapsulates all the values rug...,jacques burger prepares for battle with racing...
1274,bd86114895966473b1956b1969cd8659a095a75b,gemma collins has expanded her popular range o...,"gemma, 34, gives femail a peek at her extended..."
7101,1eba9176e136bf7f79b38974120b3cebbceec190,two mothers who dramatically rescued a baby an...,two mothers held the childrens heads above wat...
4765,3259c520201758e0b94a8018a431407e73d8ff45,lewis hamilton has come under fire for sprayin...,lewis hamilton won the chinese grand prix in s...


In [22]:
#expanding contractions
val_data.highlights = val_data.highlights.apply(expand_contractions)
val_data.article = val_data.article.apply(expand_contractions)
val_data.sample(5)

,id,article,highlights
11000,45d4016291437809aefb3404c43352539852d8b4,"in the small alaskan town of whittier, which s...","in this small alaskan town, almost all 218 res..."
3706,19e754172ca7db8c48f30729d5684cd221f23518,"jenny ramage, 35, had nine organs removed in a...","jenny ramage, 35, repeatedly complained of pai..."
5249,4fb39aba88f0a081d6d67a63fe8e42f78933be2f,archaeologists are to start digging for anothe...,archaeologists to looking for the monarch who ...
3686,87efcbdf6bb7b19f1fe6e02f3b021efba63a84a2,arsenal are searching for a fourth consecutive...,arsenal are just four points behind second-pla...
2235,4ea55c07467fa47e65e4b001926ec41757228bed,"last ary, former aston villa boss paul lambert...",phil parkinson has led bradford on several cup...


In [23]:
# Remove puncuation from word
def rm_punc_from_word(word):
    clean_alphabet_list = [
        alphabet for alphabet in word if alphabet not in string.punctuation
    ]
    return ''.join(clean_alphabet_list)

print(rm_punc_from_word('#cool!'))


# Remove puncuation from text
def rm_punc_from_text(text):
    clean_word_list = [rm_punc_from_word(word) for word in text]
    return ''.join(clean_word_list)

print(rm_punc_from_text("Frankly, my dear, I don't give a damn"))

cool
Frankly my dear I dont give a damn


In [24]:
# Remove numbers from text
def rm_number_from_text(text):
    text = re.sub('[0-9]+', '', text)
    return ' '.join(text.split())  # to rm `extra` white space

print(rm_number_from_text('You are 100times more sexier than me'))
print(rm_number_from_text('If you taught yes then you are 10 times more delusional than me'))

You are times more sexier than me
If you taught yes then you are times more delusional than me


In [27]:
import nltk

In [28]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [29]:
# Remove stopwords from text
def rm_stopwords_from_text(text):
    _stopwords = stopwords.words('english')
    text = text.split()
    word_list = [word for word in text if word not in _stopwords]
    return ' '.join(word_list)

rm_stopwords_from_text("Love means never having to say you're sorry")



'Love means never say sorry'

In [30]:
# Cleaning text
def clean_text(text):
    text = text.lower()
    text = rm_punc_from_text(text)
    text = rm_number_from_text(text)
    text = rm_stopwords_from_text(text)

    # there are hyphen(–) in many titles, so replacing it with empty str
    # this hyphen(–) is different from normal hyphen(-)
    text = re.sub('–', '', text)
    text = ' '.join(text.split())  # removing `extra` white spaces

    # Removing unnecessary characters from text
    text = re.sub("(\\t)", ' ', str(text)).lower()
    text = re.sub("(\\r)", ' ', str(text)).lower()
    text = re.sub("(\\n)", ' ', str(text)).lower()

    # remove accented chars ('Sómě Áccěntěd těxt' => 'Some Accented text')
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode(
        'utf-8', 'ignore'
    )

    text = re.sub("(__+)", ' ', str(text)).lower()
    text = re.sub("(--+)", ' ', str(text)).lower()
    text = re.sub("(~~+)", ' ', str(text)).lower()
    text = re.sub("(\+\++)", ' ', str(text)).lower()
    text = re.sub("(\.\.+)", ' ', str(text)).lower()

    text = re.sub(r"[<>()|&©ø\[\]\'\",;?~*!]", ' ', str(text)).lower()

    text = re.sub("(mailto:)", ' ', str(text)).lower()
    text = re.sub(r"(\\x9\d)", ' ', str(text)).lower()
    text = re.sub("([iI][nN][cC]\d+)", 'INC_NUM', str(text)).lower()
    text = re.sub("([cC][mM]\d+)|([cC][hH][gG]\d+)", 'CM_NUM',
                  str(text)).lower()

    text = re.sub("(\.\s+)", ' ', str(text)).lower()
    text = re.sub("(\-\s+)", ' ', str(text)).lower()
    text = re.sub("(\:\s+)", ' ', str(text)).lower()
    text = re.sub("(\s+.\s+)", ' ', str(text)).lower()

    try:
        url = re.search(r'((https*:\/*)([^\/\s]+))(.[^\s]+)', str(text))
        repl_url = url.group(3)
        text = re.sub(r'((https*:\/*)([^\/\s]+))(.[^\s]+)', repl_url, str(text))
    except Exception as e:
        pass

    text = re.sub("(\s+)", ' ', str(text)).lower()
    text = re.sub("(\s+.\s+)", ' ', str(text)).lower()

    return text

clean_text("Mrs. Robinson, you're trying to frighten, aren't you?")

'mrs robinson youre trying frighten arent'

In [32]:
train_data.article = train_data.article.apply(clean_text)
train_data.highlights = train_data.highlights.apply(clean_text)
train_data.sample(5)

KeyboardInterrupt: 